## Testing Variant callers

Here we test GATK, Sentieon, and Dragen variant callers to determine what the best mehod for variant calling is in terms of precision and sensitivity

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import re, json, os, logging, io, pprint, subprocess, requests

import pandas as pd

import vcf

from cromwell_tools.cromwell_api import CromwellAPI as cwt
from cromwell_tools import cromwell_auth
from google.cloud import storage

import cromwell_functions
from cromwell_functions import defaultCromwellRunner

In [4]:
auth_obj = cromwell_auth.CromwellAuth(url = "http://34.69.50.44:8000", header= None, auth = None)
auth_obj

You are not using any authentication with Cromwell. For security purposes, please consider adding authentication in front of your Cromwell instance!


In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/rcarter/.google/bioskryb-81ce35d92471.json"

In [6]:
storage_client = storage.Client('bioskryb')

## GATK - with Ref

Here we perform joint genotyping on 16 samples from lots 16 - 20. The samples are not subsampled, but are all similar in file size.
We first run the GATK gVCFs through GATK's haplotypeCaller, but including the subsampled Bulk2 reference sequence.

In [ ]:
gvcf_filenames = !gsutil ls gs://bioskryb-vumc-data/Full_PTA_exome_deepdive_with_other_techs/data/01-gatk_cromwell_output/**.g.vcf.gz
gvcf_filenames_n16 = [_file for _file in gvcf_filenames if re.sub(".g.vcf.gz", "", os.path.basename(_file)) in non_subsampled_sample_names] + ["gs://bioskryb-cromwell-outputs-jd7fh2/temp-folder-for-deletion/NA12878_Bulk2_merged_n450x10e6.g.vcf.gz"]

In [ ]:
df_list = [[re.sub("(.g.vcf.gz)|(_merged.+)", "", os.path.basename(_blobname)), _blobname] for _blobname in gvcf_filenames_n16]
pd.DataFrame(df_list).to_csv('tracked_data/gatk_n16_nonsubsampled_sample_map.tsv', sep = "\t", header = False, index= False)
!gsutil cp tracked_data/gatk_n16_nonsubsampled_sample_map.tsv gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/

In [ ]:
jj_input_json = cromwell_functions.download_and_read_inputs_json("gs://bioskryb_dev_wdl_and_inputs/gatk-workflows/gatk4-germline-snps-indels/2.0.0/JointGenotyping.hg38.wgs.inputs.json", storage_client)

In [ ]:
jj_input_json['JointGenotyping.sample_name_map'] = "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/gatk_n16_nonsubsampled_sample_map.tsv"
jj_input_json['JointGenotyping.callset_name'] = "nosubsampling_n16_gatk_joint_genotyping_gatk"

In [ ]:
project_name = "ComparingGatkSentieonDRAGEN"
final_output_bucket = "gs://bioskryb-work-d8f6s9"
cromwell_runs_bucket = "gs://bioskryb-dev-cromwell-runs-8dhf5d"

    
output_base_location = "{}/{}/{}".format(final_output_bucket, project_name, "04-nosubsampling_n16_joint_genotyping")
jj_options_dict = {
        "read_from_cache": False,
        "default_runtime_attributes": {
                "zones": "us-central1-a us-central1-b us-central1-c us-central1-f"
        },
        "final_workflow_outputs_dir": output_base_location,
        "use_relative_output_paths": True,
        "final_call_logs_dir": "{}/call_logs".format(output_base_location),
        "jes_gcs_root": cromwell_runs_bucket,
        "google_labels": {
                "pipeline-name": "gatk4-germline-snps-indels",
                "project-name": "comparing-gatk-sentieon-dragen"
        }
}

In [ ]:
input_iobytes = io.BytesIO(json.dumps(jj_input_json).encode())
options_iobytes = io.BytesIO(json.dumps(jj_options_dict).encode())
jj_resp = cwt.submit(auth_obj, 
       wdl_file = cromwell_functions.get_wdl_iobytes("gs://bioskryb_dev_wdl_and_inputs/gatk-workflows/gatk4-germline-snps-indels/2.0.0/JointGenotyping.wdl", storage_client), 
       inputs_files = input_iobytes,
       options_file = options_iobytes
)

In [ ]:
jj_resp.content

In [ ]:
gvcf_filenames_n16

## Sentieon - gVCF generation

The same 16 samples from above (plus the reference) are joint genotyped using Sentieon

First, generate gVCFs

In [ ]:
r1_fastq_files = !gsutil ls gs://bioskryb-vumc-data/Nova181_H2VMKDSXY/*_R1_*fastq.gz
r2_fastq_files = !gsutil ls gs://bioskryb-vumc-data/Nova182_H2VGNDSXY/*_R1_*fastq.gz
all_r1_fastqs = r1_fastq_files + r2_fastq_files

In [ ]:
all_r1_fastqs
valid_fastq_r1_filenames_n16 = [_file for _file in all_r1_fastqs if re.sub("_S[01-9].*.fastq.gz", "", os.path.basename(_file)) in non_subsampled_sample_names] + ["gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/NA12878_Bulk2_merged_n450x10e6_R1_001.fastq.gz"]
valid_fastq_r1_filenames_n16 = sorted(valid_fastq_r1_filenames_n16)

In [ ]:
sample_to_fastq_dict = {}
for _file in valid_fastq_r1_filenames_n16:
    sample_name = re.sub("_S[01-9].*.fastq.gz", "", os.path.basename(_file))
    try:
        sample_to_fastq_dict[sample_name].append(_file)
    except:
        sample_to_fastq_dict[sample_name] = [_file]

In [ ]:
sentieon_input_dict = {
  "FQ1": "",
  "FQ2": "",
  "REF": "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta",
  "READGROUP": "",
  "OUTPUT_BUCKET": "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/03-joint_genotyping_input_gvcfs/sentieon/{}",
  "BQSR_SITES": "gs://bioskryb-dev-resources-4j2g6d/Mills_and_1000G_gold_standard.indels.hg38.vcf.gz,gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.known_indels.vcf.gz,gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.dbsnp138.vcf.gz",
  "DBSNP": "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.dbsnp138.vcf.gz",
  "GVCF_OUTPUT": "true",
  "PREEMPTIBLE_TRIES": "2",
  "NONPREEMPTIBLE_TRY": True,
  "STREAM_INPUT": "True",
  "ZONES": "us-central1-a,us-central1-b,us-central1-c,us-central1-f",
  "PROJECT_ID": "bioskryb",
  "EMAIL": "rob.carter@bioskryb.com"
}

In [ ]:
for _sample_name, file_list in sample_to_fastq_dict.items():
    rg_list = []
    for _i in range(len(file_list)):
        rg_list.append("@RG\\tID:rgid-{}\\tSM:{}\\tPL:ILLUMINA".format(_i, _sample_name))
    rg = ",".join(rg_list)
    fq1 = ",".join(file_list)
    fq2 = re.sub("_R1_", "_R2_", ",".join(file_list))
    new_input_dict = deepcopy(sentieon_input_dict)
    new_input_dict['OUTPUT_BUCKET'] = sentieon_input_dict['OUTPUT_BUCKET'].format(_sample_name)
    new_input_dict['FQ1'] = fq1
    new_input_dict['FQ2'] = fq2
    new_input_dict['READGROUP'] = rg
    json_filename = "tracked_data/{}.n16.sentieoninput.json".format(_sample_name)
    with open(json_filename, 'w') as ofh:
        json.dump(new_input_dict, ofh)

In [ ]:
%%writefile tracked_data/time_and_run_sentieon.sh
#!/usr/bin/env bash
JSON_FILE=$1
LOG_FILE=${JSON_FILE}.run.log
echo Start: $(date +'%m-%d-%Y-%H-%M-%S') > $LOG_FILE
echo Running: ~/sentieon-google-genomics/runner/sentieon_runner.py $JSON_FILE
echo $(~/sentieon-google-genomics/runner/sentieon_runner.py $JSON_FILE) >> $LOG_FILE
echo End: $(date +'%m-%d-%Y-%H-%M-%S') >> $LOG_FILE

In [ ]:
!chmod 755 tracked_data/time_and_run_sentieon.sh

In [ ]:
json_files = !ls tracked_data/*.n16.sentieoninput.json
json_files

In [ ]:
!ls tracked_data/*JW-32.n16.sentieoninput.json | parallel -j 16 --max-args 1 "tracked_data/time_and_run_sentieon.sh {}"

In [ ]:
options_dict = {
        "default_runtime_attributes": {
                "zones": "us-central1-a us-central1-b us-central1-c us-central1-f",
                "noAddress": False,
                "bootDiskSizeGb": 60
        },
}

In [ ]:
default_inputs_dict = requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.input.json").json()
ref_sequence = "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta"
ref_sequence_index = "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta.fai"
dbsnp_filename = "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.dbsnp138.vcf.gz"
dbsnp_index = filename = "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.dbsnp138.vcf.gz.tbi"
local_dbsnp_filename = os.path.basename(dbsnp_filename)
local_ref = os.path.basename(ref_sequence)
local_ref_index = os.path.basename(ref_sequence_index)

bam = "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/03-joint_genotyping_input_gvcfs/sentieon/4249-JW-32/aligned_reads/dedup.bam"
bai = "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/03-joint_genotyping_input_gvcfs/sentieon/4249-JW-32/aligned_reads/dedup.bam.bai"
recal = "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/03-joint_genotyping_input_gvcfs/sentieon/4249-JW-32/aligned_reads/recal_data.table"

haplotyper_command = "sentieon driver -t 64 -r {ref} -i {bam} -q {recal} --algo Haplotyper -d {dbsnp} --emit_mode gvcf 4249-JW-32_hc.g.vcf.gz && mv 4249-JW-32_hc.g.vcf.gz* ./output/".format(ref = os.path.basename(ref_sequence),  bam = os.path.basename(bam), recal = os.path.basename(recal), dbsnp = os.path.basename(dbsnp_filename))

wdl_iobytes = io.BytesIO(requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.wdl").text.encode())
new_sentieon_inputs_dict = deepcopy(default_inputs_dict)
new_sentieon_inputs_dict["genericworkflow.GenericTask.shell_command"] = haplotyper_command
new_sentieon_inputs_dict["genericworkflow.GenericTask.input_files"] =[ref_sequence, ref_sequence_index, dbsnp_filename, dbsnp_filename_index, bam, bai, recal]
new_sentieon_inputs_dict["genericworkflow.docker_override"] = "gcr.io/bioskryb/sentieon-201911:latest"
new_sentieon_inputs_dict["genericworkflow.GenericTask.machine_mem_gb"] = 84
new_sentieon_inputs_dict["genericworkflow.GenericTask.disk_space_gb"] = 400
new_sentieon_inputs_dict["genericworkflow.GenericTask.cpu"] = 64
wdl_sentieon_inputs_iobytes = io.BytesIO(json.dumps(new_sentieon_inputs_dict).encode())
temp_sentieon_resp = cwt.submit(auth = auth_obj, wdl_file=wdl_iobytes, inputs_files=wdl_sentieon_inputs_iobytes, options_file= io.BytesIO(json.dumps(options_dict).encode()))

In [ ]:
temp_sentieon_resp.content

### Sentieon - Joint-genotyping

In [ ]:
gvcf_sentieon_filenames = !gsutil ls gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/03-joint_genotyping_input_gvcfs/sentieon/variants/**.g.vcf.gz
gvcf_sentieon_filenames_n16 = gvcf_sentieon_filenames
gvcf_sentieon_filenames_n16

In [ ]:
len(gvcf_sentieon_filenames_n16)

In [ ]:
project_name = "ComparingGatkSentieonDRAGEN"
final_output_bucket = "gs://bioskryb-work-d8f6s9"
cromwell_runs_bucket = "gs://bioskryb-dev-cromwell-runs-8dhf5d"

    
output_base_location = "{}/{}/{}".format(final_output_bucket, project_name, "data/04-nosubsampling_n16_joint_genotyping/sentieon")
jj_sentieon_options_dict = {
        "read_from_cache": False,
        "default_runtime_attributes": {
                "zones": "us-central1-a us-central1-b us-central1-c us-central1-f",
                "noAddress": False
        },
        "final_workflow_outputs_dir": output_base_location,
        "use_relative_output_paths": True,
        "final_call_logs_dir": "{}/call_logs".format(output_base_location),
        "jes_gcs_root": cromwell_runs_bucket,
        "google_labels": {
                "pipeline-name": "sentieon-gvcftyper",
                "project-name": "comparing-gatk-sentieon-dragen"
        }
}

In [ ]:
vcf_df = pd.DataFrame([gvcf_sentieon_filenames_n16], columns= [re.sub("_hc.g.vcf.gz", "_vcf=input", os.path.basename(_filename)) for _filename in gvcf_sentieon_filenames_n16])

gvcf_sentieon_index_filenames_n16 = ["{}.tbi".format(_filename) for  _filename in gvcf_sentieon_filenames_n16]
vcf_index_df = pd.DataFrame([gvcf_sentieon_index_filenames_n16], columns= [re.sub("_hc.g.vcf.gz.tbi", "_vcf_index=input", os.path.basename(_filename)) for _filename in gvcf_sentieon_index_filenames_n16])

ref_sequence = "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta"
ref_sequence_index = "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta.fai"
dbsnp_filename = "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.dbsnp138.vcf.gz"
dbsnp_filename_index = "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.dbsnp138.vcf.gz.tbi"
other_input_files_df = pd.DataFrame([[ref_sequence, ref_sequence_index, dbsnp_filename, dbsnp_filename_index]], columns=["ref=input", "ref_index=input", "dbsnp=input", "dbsnp_index=input"])
outputs_df = pd.DataFrame([["nosubsampling_n16_sentieon_joint_genotyping_sentieon.vcf.gz", "nosubsampling_n16_sentieon_joint_genotyping_sentieon.vcf.gz.tbi"]], columns = ["out_vcf=output", "out_vcf_index=output"])

inputs_df = pd.concat([vcf_df, vcf_index_df, other_input_files_df, outputs_df], axis = 1)

variant_subcommand = "{" + "} -v {".join(_colname.split("=")[0] for _colname in  vcf_df.columns) + "}"
gvcftyper_command_template = "sentieon driver -r {ref} -t 32  --algo GVCFtyper -v " + variant_subcommand + " -d {dbsnp} {out_vcf}"
gvcftyper_command_template

In [ ]:
vcf_df

In [ ]:
options_dict = {
        "default_runtime_attributes": {
                "zones": "us-central1-a us-central1-b us-central1-c us-central1-f",
                "noAddress": False,
                "bootDiskSizeGb": 20
        },
}

inputs_dict = requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.input.json").json()
inputs_dict["genericworkflow.docker_override"] = "gcr.io/bioskryb/sentieon-201911:latest"
inputs_dict["genericworkflow.GenericTask.machine_mem_gb"] = 32
inputs_dict["genericworkflow.GenericTask.disk_space_gb"] = 200
inputs_dict["genericworkflow.GenericTask.cpu"] = 32

gctyper_dcr = defaultCromwellRunner(auth_obj, gvcftyper_command_template, inputs_df, jj_sentieon_options_dict, inputs_dict, wdl_text= requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.wdl").text)

In [ ]:
gctyper_dcr.submit_jobs()

In [ ]:
gctyper_dcr.get_status()

### Sentieion - VarCal

We next run Sentieon's SNP variant recalibration

In [ ]:
varcal_df = pd.DataFrame([
    ["gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/f7791408-5ccb-415d-94c0-72c1802d5104/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.vcf.gz", 
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/f7791408-5ccb-415d-94c0-72c1802d5104/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.vcf.gz.tbi",
     "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta",
     "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta.fai",
     "gs://gcp-public-data--broad-references/hg38/v0/hapmap_3.3.hg38.vcf.gz",
     "gs://gcp-public-data--broad-references/hg38/v0/hapmap_3.3.hg38.vcf.gz.tbi",
     "gs://gcp-public-data--broad-references/hg38/v0/1000G_omni2.5.hg38.vcf.gz",
     "gs://gcp-public-data--broad-references/hg38/v0/1000G_omni2.5.hg38.vcf.gz.tbi",
     "gs://gcp-public-data--broad-references/hg38/v0/1000G_phase1.snps.high_confidence.hg38.vcf.gz",
     "gs://gcp-public-data--broad-references/hg38/v0/1000G_phase1.snps.high_confidence.hg38.vcf.gz.tbi",
     "gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf",
     "gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf.idx",
     "snp_tranches_out.txt",
     "nosubsampling_n16_sentieon_joint_genotyping_sentieon.snp.recal.vcf.gz"
    ]
], columns= ['vcf=input', 'vcf_index=input', 'ref=input', 'ref_index=input', "hapmap=input", "hapmap_index=input", "omni=input", "omni_index=input", "onekg=input", "onekg_index=input","dbsnp=input","dbsnp_index=input", "snp_tranches_out=output", "snp_recal_vcf=output"])

varcal_command_template = "sentieon driver -r {ref} -t 6  --algo VarCal --annotation QD --annotation MQRankSum --annotation ReadPosRankSum --annotation FS --annotation MQ --annotation SOR --annotation DP -v {vcf} --var_type SNP --tranches_file {snp_tranches_out} --resource {hapmap} --resource_param hapmap,known=false,training=true,truth=true,prior=15  --resource {omni} --resource_param omni,known=false,training=true,truth=true,prior=12 --resource {onekg} --resource_param onekg,known=false,training=true,truth=false,prior=10 --resource {dbsnp} --resource_param dbsnp,known=true,training=false,truth=false,prior=7 {snp_recal_vcf}"

In [ ]:
project_name = "ComparingGatkSentieonDRAGEN"
final_output_bucket = "gs://bioskryb-work-d8f6s9"
cromwell_runs_bucket = "gs://bioskryb-dev-cromwell-runs-8dhf5d"

    
output_base_location = "{}/{}/{}".format(final_output_bucket, project_name, "data/04-nosubsampling_n16_joint_genotyping/sentieon/varcal")
jj_varcal_sentieon_options_dict = {
        "read_from_cache": False,
        "default_runtime_attributes": {
                "zones": "us-central1-a us-central1-b us-central1-c us-central1-f",
                "noAddress": False
        },
        "final_workflow_outputs_dir": output_base_location,
        "use_relative_output_paths": True,
        "final_call_logs_dir": "{}/call_logs".format(output_base_location),
        "jes_gcs_root": cromwell_runs_bucket,
        "google_labels": {
                "pipeline-name": "sentieon-gvcftyper",
                "project-name": "comparing-gatk-sentieon-dragen"
        }
}

In [ ]:
inputs_dict = requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.input.json").json()
inputs_dict["genericworkflow.docker_override"] = "gcr.io/bioskryb/sentieon-201911:latest"
inputs_dict["genericworkflow.GenericTask.machine_mem_gb"] = 6
inputs_dict["genericworkflow.GenericTask.disk_space_gb"] = 150
inputs_dict["genericworkflow.GenericTask.cpu"] = 6

snp_varcal_dcr = defaultCromwellRunner(auth_obj, varcal_command_template, varcal_df, jj_varcal_sentieon_options_dict, inputs_dict, wdl_text= requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.wdl").text)

In [ ]:
snp_varcal_dcr.submit_jobs()

In [ ]:
snp_varcal_dcr.get_status()

We next run Sentieon's indel variant recalibration

In [ ]:
indel_varcal_df = pd.DataFrame([
    ["gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/f7791408-5ccb-415d-94c0-72c1802d5104/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.vcf.gz", 
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/f7791408-5ccb-415d-94c0-72c1802d5104/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.vcf.gz.tbi",
     "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta",
     "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta.fai",
     "gs://gcp-public-data--broad-references/hg38/v0/Axiom_Exome_Plus.genotypes.all_populations.poly.hg38.vcf.gz", 
     "gs://gcp-public-data--broad-references/hg38/v0/Axiom_Exome_Plus.genotypes.all_populations.poly.hg38.vcf.gz.tbi", 
     "gs://gcp-public-data--broad-references/hg38/v0/Mills_and_1000G_gold_standard.indels.hg38.vcf.gz",
     "gs://gcp-public-data--broad-references/hg38/v0/Mills_and_1000G_gold_standard.indels.hg38.vcf.gz.tbi",
     "gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf",
     "gs://gcp-public-data--broad-references/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf.idx",
     "indel_tranches_out.txt",
     "nosubsampling_n16_sentieon_joint_genotyping_sentieon.indel.recal.vcf.gz"
    ]
], columns= ['vcf=input', 'vcf_index=input', 'ref=input', 'ref_index=input', "axiom=input", "axiom_index=input", "mills=input", "mills_index=input", "dbsnp=input","dbsnp_index=input", "indel_tranches_out=output", "indel_recal_vcf=output"])

indel_varcal_command_template = "sentieon driver  -r {ref} -t 6  --algo VarCal --annotation FS --annotation ReadPosRankSum --annotation MQRankSum --annotation QD --annotation SOR --annotation DP -v {vcf} --var_type INDEL --tranches_file {indel_tranches_out} --resource {mills} --resource_param  mills,known=false,training=true,truth=true,prior=12 --resource {axiom} --resource_param axiom,known=false,training=true,truth=false,prior=10 --resource {dbsnp} --resource_param dbsnp,known=true,training=false,truth=false,prior=2 {indel_recal_vcf}"
indel_varcal_command_template

In [ ]:
project_name = "ComparingGatkSentieonDRAGEN"
final_output_bucket = "gs://bioskryb-work-d8f6s9"
cromwell_runs_bucket = "gs://bioskryb-dev-cromwell-runs-8dhf5d"

    
output_base_location = "{}/{}/{}".format(final_output_bucket, project_name, "data/04-nosubsampling_n16_joint_genotyping/sentieon/varcal")
jj_varcal_sentieon_options_dict = {
        "read_from_cache": False,
        "default_runtime_attributes": {
                "zones": "us-central1-a us-central1-b us-central1-c us-central1-f",
                "noAddress": False
        },
        "final_workflow_outputs_dir": output_base_location,
        "use_relative_output_paths": True,
        "final_call_logs_dir": "{}/call_logs".format(output_base_location),
        "jes_gcs_root": cromwell_runs_bucket,
        "google_labels": {
                "pipeline-name": "sentieon-gvcftyper",
                "project-name": "comparing-gatk-sentieon-dragen"
        }
}

In [ ]:
inputs_dict = requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.input.json").json()
inputs_dict["genericworkflow.docker_override"] = "gcr.io/bioskryb/sentieon-201911:latest"
inputs_dict["genericworkflow.GenericTask.machine_mem_gb"] = 6
inputs_dict["genericworkflow.GenericTask.disk_space_gb"] = 150
inputs_dict["genericworkflow.GenericTask.cpu"] = 6

indel_varcal_dcr = defaultCromwellRunner(auth_obj, indel_varcal_command_template, indel_varcal_df, jj_varcal_sentieon_options_dict, inputs_dict, wdl_text= requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.wdl").text)

In [ ]:
indel_varcal_dcr.submit_jobs()

In [ ]:
indel_varcal_dcr.get_status()

### Sentieon - ApplyVarCal

In [ ]:
apply_varcal_df = pd.DataFrame([
    ["gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/f7791408-5ccb-415d-94c0-72c1802d5104/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.vcf.gz", 
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/f7791408-5ccb-415d-94c0-72c1802d5104/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.vcf.gz.tbi",
     "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta",
     "gs://bioskryb-dev-resources-4j2g6d/Homo_sapiens_assembly38.fasta.fai",
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/varcal/7af01187-7275-45a7-ab26-9b923fc8d714/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/indel_tranches_out.txt",
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/varcal/7af01187-7275-45a7-ab26-9b923fc8d714/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.indel.recal.vcf.gz",
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/varcal/7af01187-7275-45a7-ab26-9b923fc8d714/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.indel.recal.vcf.gz.tbi",
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/varcal/e8702941-94ce-430e-98b6-6cb5e1d2b25d/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/snp_tranches_out.txt",
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/varcal/e8702941-94ce-430e-98b6-6cb5e1d2b25d/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.snp.recal.vcf.gz",
     "gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/varcal/e8702941-94ce-430e-98b6-6cb5e1d2b25d/call-GenericTask/glob-eb940d6b5fb9e12ecfa084f32facbc84/nosubsampling_n16_sentieon_joint_genotyping_sentieon.snp.recal.vcf.gz.tbi",
     "nosubsampling_n16_sentieon_joint_genotyping_sentieon.indel.varcal.final.vcf.gz",
     "nosubsampling_n16_sentieon_joint_genotyping_sentieon.indel.varcal.final.vcf.gz.tbi"
    ]
], columns= ['vcf=input', 'vcf_index=input', 'ref=input', 'ref_index=input', "indel_tranches=input", "indel_recal_vcf=input", "indel_recal_index_vcf=input", "snp_tranches=input", "snp_recal_vcf=input", "snp_recal_index_vcf=input", "final_varcal_vcf=output", "final_varcal_vcf_index=output"])

apply_varcal_command_template = "sentieon driver  -r {ref} -t 6  --algo ApplyVarCal --vqsr_model var_type=SNP,recal={snp_recal_vcf},tranches_file={snp_tranches},sensitivity=99.7 --vqsr_model var_type=INDEL,recal={indel_recal_vcf},tranches_file={indel_tranches},sensitivity=99.0 --vcf {vcf} {final_varcal_vcf}"
apply_varcal_command_template

In [ ]:
project_name = "ComparingGatkSentieonDRAGEN"
final_output_bucket = "gs://bioskryb-work-d8f6s9"
cromwell_runs_bucket = "gs://bioskryb-dev-cromwell-runs-8dhf5d"

    
output_base_location = "{}/{}/{}".format(final_output_bucket, project_name, "data/04-nosubsampling_n16_joint_genotyping/sentieon/applyvarcal")
jj_applyvarcal_sentieon_options_dict = {
        "read_from_cache": False,
        "default_runtime_attributes": {
                "zones": "us-central1-a us-central1-b us-central1-c us-central1-f",
                "noAddress": False
        },
        "final_workflow_outputs_dir": output_base_location,
        "use_relative_output_paths": True,
        "final_call_logs_dir": "{}/call_logs".format(output_base_location),
        "jes_gcs_root": cromwell_runs_bucket,
        "google_labels": {
                "pipeline-name": "sentieon-gvcftyper",
                "project-name": "comparing-gatk-sentieon-dragen"
        }
}

In [ ]:
inputs_dict = requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.input.json").json()
inputs_dict["genericworkflow.docker_override"] = "gcr.io/bioskryb/sentieon-201911:latest"
inputs_dict["genericworkflow.GenericTask.machine_mem_gb"] = 6
inputs_dict["genericworkflow.GenericTask.disk_space_gb"] = 150
inputs_dict["genericworkflow.GenericTask.cpu"] = 6

apply_varcal_dcr = defaultCromwellRunner(auth_obj, apply_varcal_command_template, apply_varcal_df, jj_applyvarcal_sentieon_options_dict, inputs_dict, wdl_text= requests.get("https://raw.githubusercontent.com/gatk-workflows/generic-wdl-script/master/generic-workflow.wdl").text)

In [ ]:
apply_varcal_dcr.submit_jobs()

In [ ]:
apply_varcal_dcr.get_status()

## Measuring precision and sensitivity 

The following was performed to acquire the vcf files for Sentieon and gatk samples:

`rcarter@bioskryb-dt:scratch$ gsutil cp gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/sentieon/applyvarcal/95f90949-445b-4493-b80e-ddaf09353cda/call-GenericTask/**.vcf.gz* .`

`rcarter@bioskryb-dt:scratch$ gsutil cp gs://bioskryb-work-d8f6s9/ComparingGatkSentieonDRAGEN/data/04-nosubsampling_n16_joint_genotyping/JointGenotyping/e00ab3ad-4e37-4676-af07-8a274f08c741/call-FinalGatherVcf/attempt-2/**.vcf.gz* .`

`rcarter@bioskryb-dt:scratch$ cp  ~/BaseSpace/Projects/.id.161953803/AppSessions/.id.231671923/AppResults.239697619.joint/Files/dragen-joint.hard-filtered.vcf.gz* .`

GATK 4.1.3.0 (in Docker container) was used to isolate the SNP variants from each of the three platforms

`for i in *.vcf.gz; do PREFIX=$(echo $i | sed 's/.vcf.gz//'); gatk SelectVariants --select-type SNP  -V ${PREFIX}.vcf.gz -O ${PREFIX}.snps_only.vcf.gz; done`

The vcfs were split into indels and SNP vcfs, but were not filtered. Note that the Sentieon VCF was incorrenctly named, so it was renamed first to remove the '.indel' substring.

`for i in nosubsampling_n16_sentieon_joint_genotyping_sentieon.indel.*; do NEW_NAME=$(echo $i | sed 's/.indel//'); mv $i $NEW_NAME; done`

In [6]:
%%writefile tracked_data/run_rtg.sh
# baseline vcf
# baseline sample_name
# comparison vcf
# joined comparison vcfs
#filder suffix
# extra options
#!/usr/bin/env bash
echo $4 | sed 's/,/\n/g' | parallel --max-args=1 --jobs 8 /home/rcarter/rtg-tools/bin/rtg-tools-3.10.1-4d58eadb/rtg RTG_MEM=6G vcfeval -b $1 -c $3 --sample=${2},{} -o scratch/${2}_vs_{}-${5} -e /home/rcarter/Homo_sapiens_assembly38_n25chr.bed -t /home/rcarter/R_bioskryb_na12878_analysis/NA12878_comparison/SDF/  --squash-ploidy $6

Overwriting tracked_data/run_rtg.sh


In [7]:
!chmod 755 tracked_data/run_rtg.sh

all non-bulk samples are compared against bulk  for each of the three genotyping platforms

#### GATK

In [7]:
gatk_snps_vcf = vcf.Reader(open('scratch/nosubsampling_n16_gatk_joint_genotyping_gatk.snps_only.vcf.gz', 'rb'))
gatk_samplenames = gatk_snps_vcf.samples
gatk_samplenames

['4249-JW-10',
 '4249-JW-14',
 '4249-JW-16',
 '4249-JW-17',
 '4249-JW-18',
 '4249-JW-19',
 '4249-JW-23',
 '4249-JW-25',
 '4249-JW-27',
 '4249-JW-30',
 '4249-JW-32',
 '4249-JW-36',
 '4249-JW-38',
 '4249-JW-40',
 '4249-JW-42',
 '4249-JW-43',
 'NA12878_Bulk2']

In [13]:
gatk_joined_samplenames = ",".join(gatk_samplenames)
!tracked_data/run_rtg.sh scratch/nosubsampling_n16_gatk_joint_genotyping_gatk.snps_only.vcf.gz NA12878_Bulk2 scratch/nosubsampling_n16_gatk_joint_genotyping_gatk.snps_only.vcf.gz $gatk_joined_samplenames  n16_gatk "--vcf-score-field=INFO.AS_VQSLOD"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -4.324            3213852        3213852     142639     335394     0.9575       0.9055     0.9308
     None            3220138        3220138     144497     329108     0.9571       0.9073     0.9315

There were 8133 variants not thresholded in ROC data files due to missing or invalid AS_VQSLOD (INFO) values.
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -4.324            3303278        3303278     148482     245968     0.9570       0.9307     0.9437
     None            3310365        3310365     151675     238881     0.9562       0.9327     0.9443

There were 10270 variants not thresholded in ROC data files due to missing or in

In [8]:
!tracked_data/run_rtg.sh scratch/nosubsampling_n16_gatk_joint_genotyping_gatk.snps_only.vcf.gz NA12878_Bulk2 scratch/nosubsampling_n16_gatk_joint_genotyping_gatk.snps_only.vcf.gz 4249-JW-10,4249-JW-27 all_variants_n16_gatk "--vcf-score-field=INFO.AS_VQSLOD --all-records"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
  -14.260            3310479        3310479     196923     466513     0.9439       0.8765     0.9089
     None            3339950        3339950     234330     437042     0.9344       0.8843     0.9087

There were 8753 variants not thresholded in ROC data files due to missing or invalid AS_VQSLOD (INFO) values.
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
  -15.706            3177830        3177830     191953     599162     0.9430       0.8414     0.8893
     None            3203976        3203976     224954     573016     0.9344       0.8483     0.8893

There were 10786 variants not thresholded in ROC data files due to missing or in

#### Sentieon

In [18]:
sentieon_snps_vcf = vcf.Reader(open('scratch/nosubsampling_n16_sentieon_joint_genotyping_sentieon.varcal.final.snps_only.vcf.gz', 'rb'))
sentieon_samplenames = [_sample_name for _sample_name in sentieon_snps_vcf.samples]
sentieon_samplenames

['4249-JW-10',
 '4249-JW-14',
 '4249-JW-16',
 '4249-JW-17',
 '4249-JW-18',
 '4249-JW-19',
 '4249-JW-23',
 '4249-JW-25',
 '4249-JW-27',
 '4249-JW-30',
 '4249-JW-32',
 '4249-JW-36',
 '4249-JW-38',
 '4249-JW-40',
 '4249-JW-42',
 '4249-JW-43',
 'NA12878_Bulk2']

In [19]:
sentieon_joined_samplenames = ",".join(sentieon_samplenames)
!tracked_data/run_rtg.sh scratch/nosubsampling_n16_sentieon_joint_genotyping_sentieon.varcal.final.snps_only.vcf.gz NA12878_Bulk2 scratch/nosubsampling_n16_sentieon_joint_genotyping_sentieon.varcal.final.snps_only.vcf.gz $sentieon_joined_samplenames n16_sentieon "--vcf-score-field=INFO.VQSLOD"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -8.433            3720757        3720757          0          0     1.0000       1.0000     1.0000
     None            3720757        3720757          0          0     1.0000       1.0000     1.0000



In [12]:
!tracked_data/run_rtg.sh scratch/nosubsampling_n16_sentieon_joint_genotyping_sentieon.varcal.final.snps_only.vcf.gz NA12878_Bulk2 scratch/nosubsampling_n16_sentieon_joint_genotyping_sentieon.varcal.final.snps_only.vcf.gz 4249-JW-10,4249-JW-27 all_variants_n16_sentieon "--vcf-score-field=INFO.VQSLOD --all-records" 

 Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
-----------------------------------------------------------------------------------------------------
-61878.423            3274253        3274253     145739     554828     0.9574       0.8551     0.9034
      None            3274254        3274254     145741     554827     0.9574       0.8551     0.9034

 Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
-----------------------------------------------------------------------------------------------------
-73629.829            3410571        3410571     151850     418510     0.9574       0.8907     0.9228
      None            3410571        3410571     151850     418510     0.9574       0.8907     0.9228



#### DRAGEN

In [14]:
dragen_snps_vcf = vcf.Reader(open('scratch/dragen-joint.hard-filtered.snps_only.vcf.gz', 'rb'))
dragen_samplenames = dragen_snps_vcf.samples
#dragen_samplenames = [_sample_name for _sample_name in dragen_snps_vcf.samples if _sample_name != 'NA12878-Bulk2-merged-n450x10e6']
dragen_samplenames

['4249-JW-10_combined',
 '4249-JW-14_combined',
 '4249-JW-16_combined',
 '4249-JW-17_combined',
 '4249-JW-18_combined',
 '4249-JW-19_combined',
 '4249-JW-23_combined',
 '4249-JW-25_combined',
 '4249-JW-27_combined',
 '4249-JW-30_combined',
 '4249-JW-32_combined',
 '4249-JW-36_combined',
 '4249-JW-38_combined',
 '4249-JW-40_combined',
 '4249-JW-42_combined',
 '4249-JW-43_combined',
 'NA12878-Bulk2-merged-n450x10e6']

In [15]:
dragen_joined_samplenames = ",".join(dragen_samplenames)
!tracked_data/run_rtg.sh scratch/dragen-joint.hard-filtered.snps_only.vcf.gz NA12878-Bulk2-merged-n450x10e6 scratch/dragen-joint.hard-filtered.snps_only.vcf.gz $dragen_joined_samplenames n16_dragen  "--vcf-score-field=GQ"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
    2.000            3575453        3575453     290864     371361     0.9248       0.9059     0.9152
     None            3575468        3575468     291474     371346     0.9246       0.9059     0.9152

There were 591 variants not thresholded in ROC data files due to missing or invalid GQ (FORMAT) values.
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
    2.000            3634659        3634659     316158     312155     0.9200       0.9209     0.9204
     None            3634672        3634672     316670     312142     0.9199       0.9209     0.9204

There were 494 variants not thresholded in ROC data files due to missing or invalid GQ

In [16]:
!tracked_data/run_rtg.sh scratch/dragen-joint.hard-filtered.snps_only.vcf.gz NA12878-Bulk2-merged-n450x10e6 scratch/dragen-joint.hard-filtered.snps_only.vcf.gz 4249-JW-10_combined,4249-JW-27_combined all_variants_n16_dragen  "--vcf-score-field=GQ  --all-records"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
    1.000            3399638        3399638     491441     568635     0.8737       0.8567     0.8651
     None            3399644        3399644     491964     568629     0.8736       0.8567     0.8651

There were 529 variants not thresholded in ROC data files due to missing or invalid GQ (FORMAT) values.
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
    3.000            3501461        3501461     428364     466812     0.8910       0.8824     0.8867
     None            3534030        3534030     470214     434243     0.8826       0.8906     0.8866

There were 480 variants not thresholded in ROC data files due to missing or invalid GQ

### Gather RTG summary files

In [20]:
summary_files = !find scratch/*_{dragen,sentieon,gatk} -name summary.txt | grep -v giab | grep -v all_variants
summary_files

['scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-10_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-14_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-16_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-17_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-18_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-19_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-23_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-25_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-27_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-30_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bulk2-merged-n450x10e6_vs_4249-JW-32_combined-n16_dragen/summary.txt',
 'scratch/NA12878-Bul

In [39]:
df_list = []
for _summary_file in summary_files:
    comparison_dir_name = _summary_file.split("/")[1]
    sample_name = re.sub("^.+vs_(.+)-n16.*", "\\1", comparison_dir_name)
    sample_name = re.sub("_combined", "", sample_name)
    sample_name = re.sub("NA12878-Bulk2-merged-n450x10e6", "NA12878_Bulk2", sample_name)
    platform = re.sub("^.+-n16.", "", comparison_dir_name)
    temp_df = pd.read_csv(_summary_file, skiprows=2, sep = "[\t ]+", header = None, names=["Threshold","True-pos-baseline","True-pos-call","False-pos","False-neg","Precision","Sensitivity","F-measure"]).head(1)
    temp_df['sample_name'] = sample_name
    temp_df['platform'] = platform
    df_list.append(temp_df)
rtg_summary_df = pd.concat(df_list)
rtg_summary_df

/home/rcarter/anacon3/lib/python3.7/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,Threshold,True-pos-baseline,True-pos-call,False-pos,False-neg,Precision,Sensitivity,F-measure,sample_name,platform
0,2.000,3533845,3533845,304005,412969,0.9208,0.8954,0.9079,4249-JW-10,dragen
0,1.000,3613773,3613773,291013,333041,0.9255,0.9156,0.9205,4249-JW-14,dragen
0,1.000,3529715,3529715,285187,417099,0.9252,0.8943,0.9095,4249-JW-16,dragen
0,2.000,3575453,3575453,290864,371361,0.9248,0.9059,0.9152,4249-JW-17,dragen
0,2.000,3669511,3669511,300840,277303,0.9242,0.9297,0.9270,4249-JW-18,dragen
0,2.000,3634659,3634659,316158,312155,0.9200,0.9209,0.9204,4249-JW-19,dragen
0,2.000,3273458,3273458,294481,673356,0.9175,0.8294,0.8712,4249-JW-23,dragen
0,2.000,3655618,3655618,332261,291196,0.9167,0.9262,0.9214,4249-JW-25,dragen
0,1.000,3399462,3399462,318885,547352,0.9142,0.8613,0.8870,4249-JW-27,dragen
0,2.000,3376201,3376201,305849,570613,0.9169,0.8554,0.8851,4249-JW-30,dragen


In [40]:
rtg_summary_df.to_csv("tracked_data/rtg_summary_all_samples.tsv", sep = "\t")

In [41]:
rtg_summary_df

,Threshold,True-pos-baseline,True-pos-call,False-pos,False-neg,Precision,Sensitivity,F-measure,sample_name,platform
0,2.000,3533845,3533845,304005,412969,0.9208,0.8954,0.9079,4249-JW-10,dragen
0,1.000,3613773,3613773,291013,333041,0.9255,0.9156,0.9205,4249-JW-14,dragen
0,1.000,3529715,3529715,285187,417099,0.9252,0.8943,0.9095,4249-JW-16,dragen
0,2.000,3575453,3575453,290864,371361,0.9248,0.9059,0.9152,4249-JW-17,dragen
0,2.000,3669511,3669511,300840,277303,0.9242,0.9297,0.9270,4249-JW-18,dragen
0,2.000,3634659,3634659,316158,312155,0.9200,0.9209,0.9204,4249-JW-19,dragen
0,2.000,3273458,3273458,294481,673356,0.9175,0.8294,0.8712,4249-JW-23,dragen
0,2.000,3655618,3655618,332261,291196,0.9167,0.9262,0.9214,4249-JW-25,dragen
0,1.000,3399462,3399462,318885,547352,0.9142,0.8613,0.8870,4249-JW-27,dragen
0,2.000,3376201,3376201,305849,570613,0.9169,0.8554,0.8851,4249-JW-30,dragen


### Comparing all samples to GIAB

We compare all samples in each VCF to the GIAB sample. This gives us an objectibve means for comparison between vendors

#### GATK

In [25]:
!tracked_data/run_rtg.sh /home/rcarter/R_bioskryb_na12878_analysis/NA12878_comparison/HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz HG001 scratch/nosubsampling_n16_gatk_joint_genotyping_gatk.snps_only.vcf.gz $gatk_joined_samplenames  giab_vs_n16_gatk "--vcf-score-field=INFO.AS_VQSLOD"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -1.811            2834581        2834579     499181     784890     0.8503       0.7831     0.8153
     None            2872912        2872878     575870     746559     0.8330       0.7937     0.8129

Reference sequence chrY is used in calls but not in baseline.
There were 9860 variants not thresholded in ROC data files due to missing or invalid AS_VQSLOD (INFO) values.
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -1.923            2708277        2708276     474304     911194     0.8510       0.7483     0.7963
     None            2742167        2742135     545104     877304     0.8342       0.7576     0.7941

Reference sequence

In [26]:
!tracked_data/run_rtg.sh /home/rcarter/R_bioskryb_na12878_analysis/NA12878_comparison/HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz HG001 scratch/nosubsampling_n16_gatk_joint_genotyping_gatk.snps_only.vcf.gz 4249-JW-10,4249-JW-27 all_variants_giab_vs_n16_gatk "--vcf-score-field=INFO.AS_VQSLOD --all-records"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -1.923            2771796        2771790     486836     847676     0.8506       0.7658     0.8060
     None            2844845        2844810     729470     774626     0.7959       0.7860     0.7909

Reference sequence chrY is used in calls but not in baseline.
There were 8753 variants not thresholded in ROC data files due to missing or invalid AS_VQSLOD (INFO) values.
Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -1.923            2656335        2656334     468844     963136     0.8500       0.7339     0.7877
     None            2727095        2727062     701868     892376     0.7953       0.7535     0.7738

Reference sequence

#### Sentieon

In [27]:
!tracked_data/run_rtg.sh /home/rcarter/R_bioskryb_na12878_analysis/NA12878_comparison/HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz HG001 scratch/nosubsampling_n16_sentieon_joint_genotyping_sentieon.varcal.final.snps_only.vcf.gz $sentieon_joined_samplenames giab_vs_n16_sentieon "--vcf-score-field=INFO.VQSLOD"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -1.725            2735036        2735007     480093     884435     0.8507       0.7556     0.8003
     None            2788006        2787973     611920     831465     0.8200       0.7703     0.7944

Reference sequence chrY is used in calls but not in baseline.
Reference sequence chrM is used in calls but not in baseline.
Reference sequence chr1_KI270706v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270707v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270708v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270709v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270710v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270711v1_random is used in calls but not in b

In [28]:
!tracked_data/run_rtg.sh /home/rcarter/R_bioskryb_na12878_analysis/NA12878_comparison/HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz HG001 scratch/nosubsampling_n16_sentieon_joint_genotyping_sentieon.varcal.final.snps_only.vcf.gz 4249-JW-10,4249-JW-27 all_variants_giab_vs_n16_sentieon "--vcf-score-field=INFO.VQSLOD --all-records" 

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
   -1.726            2787734        2787704     488648     831737     0.8509       0.7702     0.8085
     None            2847544        2847506     714915     771927     0.7993       0.7867     0.7930

Reference sequence chrY is used in calls but not in baseline.
Reference sequence chrM is used in calls but not in baseline.
Reference sequence chr1_KI270706v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270707v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270708v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270709v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270710v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270711v1_random is used in calls but not in b

#### DRAGEN

In [30]:
!tracked_data/run_rtg.sh /home/rcarter/R_bioskryb_na12878_analysis/NA12878_comparison/HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz HG001 scratch/dragen-joint.hard-filtered.snps_only.vcf.gz $dragen_joined_samplenames giab_vs_n16_dragen  "--vcf-score-field=GQ"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
    5.000            2855900        2855861     857036     763571     0.7692       0.7890     0.7790
     None            2890792        2890754     947599     728679     0.7531       0.7987     0.7752

Reference sequence chrY is used in calls but not in baseline.
Reference sequence chrM is used in calls but not in baseline.
Reference sequence chr1_KI270706v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270707v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270708v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270709v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270710v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270711v1_random is used in calls but not in b

In [34]:
!tracked_data/run_rtg.sh /home/rcarter/R_bioskryb_na12878_analysis/NA12878_comparison/HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz HG001 scratch/dragen-joint.hard-filtered.snps_only.vcf.gz 4249-JW-10_combined,4249-JW-27_combined all_variants_giab_vs_n16_dragen  "--vcf-score-field=GQ --all-records"
#!tracked_data/run_rtg.sh scratch/dragen-joint.hard-filtered.snps_only.vcf.gz NA12878-Bulk2-merged-n450x10e6 scratch/dragen-joint.hard-filtered.snps_only.vcf.gz 4249-JW-10_combined,4249-JW-27_combined all_variants_n16_dragen  "--vcf-score-field=GQ  --all-records"

Threshold  True-pos-baseline  True-pos-call  False-pos  False-neg  Precision  Sensitivity  F-measure
----------------------------------------------------------------------------------------------------
    5.000            2736619        2736590     994861     882852     0.7334       0.7561     0.7446
     None            2780070        2780039    1111569     839401     0.7144       0.7681     0.7403

Reference sequence chrY is used in calls but not in baseline.
Reference sequence chrM is used in calls but not in baseline.
Reference sequence chr1_KI270706v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270707v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270708v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270709v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270710v1_random is used in calls but not in baseline.
Reference sequence chr1_KI270711v1_random is used in calls but not in b

### Gather GIAB-comparison RTG summary files

In [35]:
summary_giab_files = !find scratch/HG001*-giab_vs_n16_{dragen,sentieon,gatk} -name summary.txt
summary_giab_files

['scratch/HG001_vs_4249-JW-10_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-14_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-16_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-17_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-18_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-19_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-23_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-25_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-27_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-30_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-32_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-36_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-38_combined-giab_vs_n16_dragen/summary.txt',
 'scratch/HG001_vs_4249-JW-40_combined-giab_vs_n16_dragen/summar

In [36]:
df_list = []
for _summary_file in summary_giab_files:
    comparison_dir_name = _summary_file.split("/")[1]
    sample_name = re.sub("HG001_vs_(.+)-giab_.+", "\\1", comparison_dir_name)
    sample_name = re.sub("_combined", "", sample_name)
    sample_name = re.sub("NA12878-Bulk2-merged-n450x10e6", "NA12878_Bulk2", sample_name)
    platform = re.sub("^.+_", "", comparison_dir_name)
    temp_df = pd.read_csv(_summary_file, skiprows=2, sep = "[\t ]+", header = None, names=["Threshold","True-pos-baseline","True-pos-call","False-pos","False-neg","Precision","Sensitivity","F-measure"]).tail(1)
    temp_df['sample_name'] = sample_name
    temp_df['platform'] = platform
    df_list.append(temp_df)
rtg_giab_summary_df = pd.concat(df_list)
rtg_giab_summary_df

/home/rcarter/anacon3/lib/python3.7/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,Threshold,True-pos-baseline,True-pos-call,False-pos,False-neg,Precision,Sensitivity,F-measure,sample_name,platform
1,None,2890792,2890754,947599,728679,0.7531,0.7987,0.7752,4249-JW-10,dragen
1,None,2956463,2956426,948866,663008,0.7570,0.8168,0.7858,4249-JW-14,dragen
1,None,2887412,2887374,928012,732059,0.7568,0.7977,0.7767,4249-JW-16,dragen
1,None,2918886,2918857,948085,700585,0.7548,0.8064,0.7798,4249-JW-17,dragen
1,None,2999233,2999200,971792,620238,0.7553,0.8286,0.7903,4249-JW-18,dragen
1,None,2972312,2972274,979068,647159,0.7522,0.8212,0.7852,4249-JW-19,dragen
1,None,2668004,2667967,900548,951467,0.7476,0.7371,0.7423,4249-JW-23,dragen
1,None,2996862,2996818,991578,622609,0.7514,0.8280,0.7878,4249-JW-25,dragen
1,None,2780062,2780031,938845,839409,0.7475,0.7681,0.7577,4249-JW-27,dragen
1,None,2760285,2760250,922241,859186,0.7496,0.7626,0.7560,4249-JW-30,dragen


In [37]:
rtg_giab_summary_df.to_csv("tracked_data/rtg_summary_giab_vs_all_samples.tsv", sep = "\t")

In [38]:
rtg_giab_summary_df

,Threshold,True-pos-baseline,True-pos-call,False-pos,False-neg,Precision,Sensitivity,F-measure,sample_name,platform
1,None,2890792,2890754,947599,728679,0.7531,0.7987,0.7752,4249-JW-10,dragen
1,None,2956463,2956426,948866,663008,0.7570,0.8168,0.7858,4249-JW-14,dragen
1,None,2887412,2887374,928012,732059,0.7568,0.7977,0.7767,4249-JW-16,dragen
1,None,2918886,2918857,948085,700585,0.7548,0.8064,0.7798,4249-JW-17,dragen
1,None,2999233,2999200,971792,620238,0.7553,0.8286,0.7903,4249-JW-18,dragen
1,None,2972312,2972274,979068,647159,0.7522,0.8212,0.7852,4249-JW-19,dragen
1,None,2668004,2667967,900548,951467,0.7476,0.7371,0.7423,4249-JW-23,dragen
1,None,2996862,2996818,991578,622609,0.7514,0.8280,0.7878,4249-JW-25,dragen
1,None,2780062,2780031,938845,839409,0.7475,0.7681,0.7577,4249-JW-27,dragen
1,None,2760285,2760250,922241,859186,0.7496,0.7626,0.7560,4249-JW-30,dragen
